SCRIPT CRS: EPSG:32632

In [ ]:
import geopandas as gpd
import numpy as np
import os
import rasterio
from rasterio.features import geometry_mask
from scipy.ndimage import distance_transform_edt
from rasterio.plot import show
import matplotlib.pyplot as plt
from rasterio.features import rasterize
from shapely.geometry import box
from scipy.ndimage import convolve

import functions as fun

# Load DEM
First I load the DEM since it is the base of all maps I will create.

In [ ]:
with rasterio.open('../data/terrain/tinitaly_dem10m.tif') as src:
    dem = src.read(1)  # DEM data
    dem_transform = src.transform  # Affine transform for georeferencing
    dem_crs = src.crs 
    dem_profile = src.profile
    dem_shape = src.shape
    dem_bounds = src.bounds  # (left, bottom, right, top)


# 1. Accessibility
## 1.1 Distance from trails
Compute the eucledian distance from the nearest hiking trail for each cell in the DEM. Then, standardize it so that the value is 1 when the distance is zero and decreases until becoming zero for distances of more than 500m.

In [ ]:
# Load trails gdf
trails_gdf = gpd.read_file('../data/trails/Sentieri_della_SAT.shp')
trails_gdf = trails_gdf.to_crs(dem_crs)

In [ ]:
# Create a binary mask where cells with trails are False
trail_mask = geometry_mask([geom for geom in trails_gdf.geometry], 
                           out_shape=dem.shape,
                           transform=dem_transform, 
                           invert=True)

# Compute the Euclidean distance transform on the binary mask
distance_array = distance_transform_edt(~trail_mask) * src.res[0]
standardized_distance = np.clip(1 - (distance_array / 500), 0, 1)


In [ ]:
# Plot trail distance map
fig, ax = plt.subplots(figsize=(10, 10))
show(standardized_distance, transform=dem_transform, cmap='YlGn', ax=ax)
plt.colorbar(ax.imshow(standardized_distance, cmap='YlGn'), ax=ax, label='Normalized Distance to Nearest Trail')
ax.set_title("Proximity to Hiking Trails")
plt.show()

In [ ]:
# Save raster map
#fun.save_raster_map(standardized_distance, dem_profile, filename='standardized_distance_trails')

## 1.2 Hut's density
For each cell, count the number of huts within a 50m buffer. The results are standardized so that a value of 1 indicates the maximum number of huts observed for a cell and 0 is where there are no huts.

In [ ]:
def compute_density(points_gdf, radius=10, per_km2=True, dem_shape=dem_shape, dem_transform=dem_transform):
    ''' Compute density of points (water sources, huts, etc..) by rasterizing them.

    Parameters:
    - points_gdf: gdf containing points
    - radius: radius of kernel [number of cells]
    - per_mk2: if True, return density per km2, else return density per kernel area

    Returns:
    - density np.ndarray 
    '''
    # Rasterize water (binary raster)
    points_raster = rasterize(
        [(geom, 1) for geom in points_gdf.geometry],
        out_shape=dem_shape,
        transform=dem_transform,
        fill=0,
        dtype='float'
    )
    print(f"Number of cells: {np.sum(points_raster==0)}")
    print(f"Cells with water: {np.sum(points_raster==1)}")

    # Create a circular kernel
    kernel = circular_kernel(radius)
    resolution = 10 # not changeable
    cell_area = resolution**2  # area of one cell [m²]
    kernel_area = kernel.sum() * cell_area  # total area covered by the kernel [m²]

    # Apply convolution to compute number of springs per kernel area
    density_raster = convolve(points_raster, kernel, mode='constant', cval=0.0)
    if np.all(density_raster == 0):
        raise ValueError("Density is all zero, check for errors.")
        
    # Convert to springs per km²
    density_raster_km2 = density_raster * (1_000_000 / kernel_area)

    return density_raster_km2 if per_km2 else density_raster

def circular_kernel(radius):
    y, x = np.ogrid[-radius:radius+1, -radius:radius+1]
    mask = x**2 + y**2 <= radius**2
    return mask.astype(float) # sum of values in the area

In [ ]:
# Load huts
huts_gdf = gpd.read_file('../data/huts/huts_points.geojson')
huts_gdf = huts_gdf.to_crs(dem_crs)

In [ ]:
huts_density_km2 = compute_density(huts_gdf)
# Optional: Normalize the density
normalized_density_raster = huts_density_km2 / huts_density_km2.max()

# 2. Environmental quality
## 2.1 Distance from protected areas
For each cell, compute the distance from the closest natural park. Results are standardized so a value of 1 indicates distance of 0m and decreases until zero for a distance >= 1km. 

In [ ]:
# Load protected areas polygons
natural_parks = gpd.read_file('../data/others/nat_parks/z307_p_pup.shp')
natural_parks = natural_parks.to_crs(dem_crs)

In [ ]:
# Create a binary mask where cells with trails are False
park_mask = geometry_mask([geom for geom in natural_parks.geometry], 
                           out_shape=dem.shape,
                           transform=dem_transform, 
                           invert=True)

# Compute the Euclidean distance transform on the binary mask
distance_array = distance_transform_edt(~park_mask) * src.res[0]
standardized_distance = np.clip(1 - (distance_array / 1000), 0, 1)

In [ ]:
# Save raster map
#fun.save_raster_map(standardized_distance, dem_profile, filename='standardized_distance_parks')

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
show(standardized_distance, transform=dem_transform, cmap='YlGn', ax=ax)
plt.colorbar(ax.imshow(standardized_distance, cmap='YlGn'), ax=ax, label='Normalized Distance to Nearest Natural Park')
ax.set_title("Proximity to Natural Parks")
plt.show()

# Distance from superficial waters

In [ ]:
def load_water_data(path, dem_crs=dem_crs, dem_bounds=dem_bounds): 
    water_gdf = gpd.read_file(path)
    water_gdf = water_gdf.to_crs(dem_crs)
    # Clip using DEM
    dem_bbox = box(dem_bounds.left, dem_bounds.bottom, dem_bounds.right, dem_bounds.top)
    clipped_gdf = water_gdf[water_gdf.geometry.within(dem_bbox)]
    print(f"Number of water points: {len(clipped_gdf)}")
    return clipped_gdf

In [ ]:
# Load data
superficial_gdf = load_water_data('../data/water/carta_ris_idriche/pup_as/pup_as.shp')
# Compute density
density_raster_km2 = compute_density(superficial_gdf, per_km2=True)
# Optional: Normalize the density 
normalized_density_raster = density_raster_km2 / density_raster_km2.max()

In [ ]:
# Save raster map
#fun.save_raster_map(normalized_density_raster, dem_profile, filename='norm_density_superficialwaters_km2')

# Distance from springs 

In [ ]:
# Load data
springs_gdf = load_water_data('../data/water/carta_ris_idriche/pup_so/pup_so.shp')
# Compute density
density_raster_km2 = compute_density(springs_gdf, per_km2=True)
# Optional: Normalize the density
normalized_density_raster = density_raster_km2 / density_raster_km2.max()

In [ ]:
# Save raster map
#fun.save_raster_map(normalized_density_raster, dem_profile, filename='norm_density_springs_km2')

In [ ]:
# Assuming spring_raster is a 2D NumPy array
plt.figure(figsize=(10, 8))
plt.imshow(density_raster_km2, cmap='binary_r', interpolation='nearest')
plt.colorbar(label='Normalized Spring Presence')
plt.title('Water Springs Density')
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.show()

# Distance from water-wells

In [ ]:
# Load data
wells_gdf = load_water_data('../data/water/carta_ris_idriche/pup_po/pup_po.shp')
# Compute density
density_raster_km2 = compute_density(wells_gdf, per_km2=True)
# Optional: Normalize the density 
normalized_density_raster = density_raster_km2 / density_raster_km2.max()

In [ ]:
# Save raster map
#fun.save_raster_map(normalized_density_raster, dem_profile, filename='norm_density_waterwells_km2')